This notebook takes you through the necessary data transformations to produce a basic COVID-19 dashboard using data available on the Johns Hopkins Github page and through the Twitter API.  

In [2]:
import os
import pandas as pd
from zipfile import ZipFile
import shutil
import numpy as np
from twython import Twython
import json
import datetime as dt

If this is your first time running this notebook you can skip the next block of code. This block simply deletes old data from the last time I updated the dashboard. 

In [4]:
os.remove("C:\\Users\\zacha\\Downloads\\COVID-19-master.zip") # replace with your file path
shutil.rmtree("C:\\Users\\zacha\\covid_dashboard") # replace with your file path

The data is available for download at this link https://github.com/CSSEGISandData/COVID-19 The block of code below just creates the directory where you want to store your data. Update the file paths as necessary. 

In [5]:
os.chdir("C:\\Users\\zacha")
!mkdir covid_dashboard


zippedfile = "C:\\Users\\zacha\\Downloads\\COVID-19-master.zip"
destination_directory = "C:\\Users\\zacha\\covid_dashboard"

with ZipFile(zippedfile, 'r') as zipObj:
   zipObj.extractall(destination_directory)

os.chdir("C:\\Users\\zacha\\covid_dashboard\\COVID-19-master\\csse_covid_19_data\\csse_covid_19_time_series")

The following block will make a table that will allow you to create a visual where it is possible to compare countries on different metrics. For example, we could look at confirmed cases, deaths, and recoveries for just the U.S. or put the U.S., Italy, and Spain all on the same graph at the same time. Since the standard Power BI line chart does not support multiple legends (i.e., different linetypes for deaths, cases, and recoveries and different colors for each country) we'll create a column called "measure" which combines both the country/region and the virus metric. Johns Hopkins has a made a few changes with file names and how data is reported as the situation has evolved, but this is current as of mid-April. 

In [6]:
# loop through the 3 different files we are pulling data from
files = ['confirmed_global', 'deaths_global', 'recovered_global']

dataframes = []
for a_file in files:
    
    df = pd.read_csv("time_series_covid19_" + a_file + ".csv")
    
    # Australia, China, and Canada have province level data but do not have a row for whole country totals
    df = df[df['Province/State'].isna() | df['Country/Region'].isin(['Australia', 'China', 'Canada'])]

    df = pd.melt(df, id_vars =['Province/State', 'Country/Region', 'Lat', 'Long'],
             var_name ='Date', value_name = a_file[:-7])
    
    dataframes.append(df)

df = dataframes[0].merge(dataframes[1], how = 'left', on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'])\
                  .merge(dataframes[2], how = 'left', on = ['Province/State', 'Country/Region', 'Lat', 'Long', 'Date'])

# sum Australian, Chinese, and Canadian data to get country totals
df2 = df[df['Country/Region'].isin(['Australia', 'China', 'Canada'])]\
        .groupby(['Country/Region', 'Date'])\
        .sum()\
        .reset_index()\
        .drop(columns = ['Lat', 'Long'])

df = df[df['Province/State'].isna()]

df.drop(columns = ['Province/State', 'Lat', 'Long'], inplace = True)

df = df.append(df2)

df = pd.melt(df, id_vars = ['Country/Region', 'Date'], var_name = 'Measure', value_name = 'Total')

df.loc[df.Measure == 'confirmed', 'Measure'] = "Confirmed Cases"
df.loc[df.Measure == 'deaths', 'Measure'] = "Deaths"
df.loc[df.Measure == 'recovered', 'Measure'] = "Recoveries"

df['Country/Region and Measure'] = df['Country/Region'] + " - " + df['Measure']

# we'll calculate a "daily increase" in deaths/cases/recoveries for each day and country for an additional visual
df['Date'] = pd.to_datetime(df['Date'])

df['Previous_Day_Total'] = df.sort_values(by=['Country/Region', 'Measure', 'Date'], ascending=True)\
                             .groupby(['Country/Region', 'Measure', 'Country/Region and Measure'])['Total']\
                             .shift(1)

df['Previous_Day_Total'] = df['Previous_Day_Total'].fillna(0)

df['Daily Increase'] = df['Total'] - df['Previous_Day_Total']

df.drop(columns = ['Previous_Day_Total'], inplace = True)


df.to_csv('joined_global_time_series.csv')

This block gets current country totals which allows you to easily make a world map. 

In [8]:
df_current = df[df.Date == max(df.Date)]\
    .drop(columns = ['Country/Region and Measure', 'Date', 'Daily Increase'])\
    .pivot_table(values='Total', index='Country/Region', columns='Measure')\
    .reset_index()

df_current['Natural Log of Current Cases'] = np.log(df_current['Confirmed Cases'] + 1)


df_current.to_csv('current_global_cases.csv')

This section will produce totals at a state level and the one day change. This will allow you to make a table similar to what is on the "worldometer" COVID website.  

In [9]:
df_USc = pd.read_csv("time_series_covid19_confirmed_US.csv")
df_USc.drop(columns = ['UID', 'code3', 'FIPS', 'Lat', 'Long_'], inplace = True)
df_USc = df_USc.groupby(['Province_State', 'Country_Region']).sum().reset_index()
df_USc.rename(columns = {df_USc.columns[-1]: "Confirmed Cases",
                         df_USc.columns[-2]: "Yesterday Confirmed Cases"}, inplace = True)
df_USc = df_USc[['Province_State', 'Confirmed Cases', 'Yesterday Confirmed Cases']]
df_USc['New Cases Yesterday'] = df_USc['Confirmed Cases'] - df_USc['Yesterday Confirmed Cases']

df_USd = pd.read_csv("time_series_covid19_deaths_US.csv")
df_USd.drop(columns = ['UID', 'code3', 'FIPS', 'Lat', 'Long_'], inplace = True)
df_USd = df_USd.groupby(['Province_State', 'Country_Region']).sum().reset_index()
df_USd.rename(columns = {df_USd.columns[-1]: "Deaths",
                         df_USd.columns[-2]: "Yesterday Deaths"}, inplace = True)
df_USd = df_USd[['Province_State', 'Deaths', 'Yesterday Deaths']]
df_USd['New Deaths Yesterday'] = df_USd['Deaths'] - df_USd['Yesterday Deaths']


df_US = pd.merge(df_USc, df_USd, how = 'left')
df_US = df_US[['Province_State', 'Confirmed Cases', 'New Cases Yesterday', 'Deaths', 'New Deaths Yesterday']]
df_US.to_csv('current_US_counts.csv')

This section will allow you to make a filled map with county level data. I've filtered by FIPS to remove some data that would work using a dot plot map (cruise ships) but doesn't work well with a filled map. The key to getting Power BI to recognize all of the county names correctly when you place them into the location field is to combine info into a format like "Anoka County, Minnesota" and to realize that the state of Louisiana has parishes and not counties. If you skip that piece of formating with the standard map you'll run into issues with common county names like Cook County, Georgia and Cook County, Minnesota. 

In [10]:
df = pd.read_csv("time_series_covid19_confirmed_US.csv")
df = df[(df.FIPS > 1000) &(df.FIPS < 57000)]
df.loc[df.Province_State == 'Louisiana', 'County'] = df.loc[df.Province_State == 'Louisiana', 'Admin2'] + ' Parish, ' +  df.loc[df.Province_State == 'Louisiana', 'Province_State']
df.loc[df.Province_State != 'Louisiana', 'County'] = df.loc[df.Province_State != 'Louisiana', 'Admin2'] + ' County, ' +  df.loc[df.Province_State != 'Louisiana', 'Province_State']
df = df.iloc[:,-2:]
df.rename(columns = {df.columns[0]: 'Confirmed Cases'}, inplace = True)
df_cc = df


df = pd.read_csv("time_series_covid19_deaths_US.csv")
df = df[(df.FIPS > 1000) &(df.FIPS < 57000)]
df.loc[df.Province_State == 'Louisiana', 'County'] = df.loc[df.Province_State == 'Louisiana', 'Admin2'] + ' Parish, ' +  df.loc[df.Province_State == 'Louisiana', 'Province_State']
df.loc[df.Province_State != 'Louisiana', 'County'] = df.loc[df.Province_State != 'Louisiana', 'Admin2'] + ' County, ' +  df.loc[df.Province_State != 'Louisiana', 'Province_State']
df = df.iloc[:,-2:]
df.rename(columns = {df.columns[0]: 'Deaths'}, inplace = True)

df = pd.merge(df, df_cc)

df['Natural Log of Current Cases'] = np.log(df.loc[:,'Confirmed Cases'] + 1)

df.to_csv('county_data.csv')

I chose to pull some tweets to create a replica twitter timeline using the "Timeline by Cloudscope" visual. In order to pull from Twitter you must first set up a developer account. I've chosen to use the Twython package but a number of alternatives exist. Tweepy is quite popular. 

In [11]:
python_tweets = Twython("Your Twitter API key", "Your secret key")

query = {'q': 'covid',
        'result_type': 'mixed',
        'count': 100,
        'lang': 'en',
        }

results = python_tweets.search(**query)

dict_ = {'user': [],'name': [], 'date': [], 'text': [], 'profile_picture': [],
         'favorite_count': [], 'retweet_count': [],'tweetJSON': [], 'truncated': []}

for status in results['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['name'].append(status['user']['name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['profile_picture'].append(status['user']['profile_image_url'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['tweetJSON'].append(json.dumps(status))
    dict_['truncated'].append(status['truncated'])
    
df_twitter = pd.DataFrame(dict_)

df_twitter['formated_date'] = pd.to_datetime(df_twitter.date).dt.strftime('%m/%d/%Y')

df_twitter.to_csv('covid_tweets.csv')